#### Пайплайн для классификации новых текстов для модели rubert-tiny2

In [1]:
import re
import torch

In [2]:
from utils import *

In [3]:
from transformers import logging
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [4]:
def clean(s):
    s = re.sub('\n', ' ', s)
    s = re.sub('<\?xml.+\?>', ' ', s)
    s = re.sub('<\/?[A-Za-z]*>', ' ', s)
    s = re.sub('[0-9]+(\.|,)?[0-9]*', ' num ', s)
    s = re.sub('([a-zа-яёй])([A-ZА-ЯЁЙ])', '\g<1> \g<2>', s)
    s = re.sub('[^A-Za-zА-ЯЁЙа-яёй0-9.,;:!?()"\s\-\']', ' ', s)
    return re.sub('\s{2,}', ' ', s).strip()

* *ничем не отличается от функции из 7ой тетрадки*

In [5]:
def tokenize(title, text, tokenizer):
    
    params = {'max_length': 512, 
              'truncation': True, 
              'return_tensors': 'pt', 
              'add_special_tokens': True}
    
    title, text = clean(title), clean(text)
    return tokenizer.encode(title, text, **params)

* *функция токенизации одного примера (title + text)*

In [6]:
def load_model(name, path):
    
    params = {
        'num_labels': 6, 
        'output_attentions': False, 
        'output_hidden_states': True
    }
    
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModelForSequenceClassification.from_pretrained(name, **params)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    return model, tokenizer

* *загрузка предобученной модели + загрузка лучших весов*

In [7]:
def predict_proba(title, text, model, tokenizer):
    
    model = model.eval()
    torch.set_grad_enabled(False)
    input_ids = tokenize(title, text, tokenizer)
    attn_mask = (input_ids != tokenizer.pad_token_id)
    
    params = {
        'output_attentions': False, 
        'output_hidden_states': False, 
        'attention_mask': attn_mask, 
        'input_ids': input_ids
    }
    
    output = model(**params)
    logits = output['logits']
    probas = torch.softmax(logits, 1)
    return probas[0].round(decimals=3)

* *функция для получения распределения вероятностей для одного примера*

In [8]:
logging.set_verbosity_error()

In [9]:
model, tokenizer = load_model('cointegrated/rubert-tiny2', 'data/rubert2.pt')

* *загружаем предобученную модель и токенизатор, подгружаем лучшие веса*

In [10]:
title = 'В декабре SpaceX, вероятно, наконец-то отправит Starship в первый орбитальный полет'
text = '''Сегодня SpaceX впервые за три года запускает ракету-носитель сверхтяжелого класса 
Falcon Heavy с миссией USSF-44, которая доставит на геостационарную орбиту 6 полезных нагрузок 
для Космических сил США (расчетное время старта — 15:41 по киевскому времени). Но есть запуск, 
которого мы ждем еще больше и уже достаточно давно. Речь идет о первом орбитальном полете 
космического корабля нового поколения Starship — в NASA предполагают, что с большой вероятностью 
запуск состоится уже в декабре этого года. По крайней мере, такова новая актуальная оценка, 
которой поделился Марк Кирасич, глава консультативного комитета пилотируемых космических полетов 
NASA и заместитель администратора по развитию амбициозной американской лунной программы 
«Артемида» / Artemis, во время недавнего заседания.'''

In [11]:
%time predict_proba(title, text, model, tokenizer)

CPU times: user 72.2 ms, sys: 7.11 ms, total: 79.3 ms
Wall time: 45.5 ms


tensor([0.0010, 0.0000, 0.0060, 0.9930, 0.0000, 0.0000])

In [12]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']

* *нужно более подробно тестировать производительность для текстов различной длины*
* *на данном конкретном примере скорость работы фактически такая же, как у catboost*
* *а это всего в 2а раза медленнее, чем самая быстрая модель на logreg*
* *все модели запускались на CPU (локально, на моем пк)*
* *есть потенциал для ускорения (каскадирование)*